This notebook presents an application of the [pydiva2D](./pydiva2d.py) module to perform a 2D Diva analysis and then generate figures for the different input and output using the Folium module.<br>

**Note:** since this module accesses various leaflet libraries hosted on CDNs, an internet connection is needed to have a working example. If you're familiar with leaflet, a workaround consist in editing the generated map in html and replace the URLs to the libraries with local paths.

In [1]:
import os
import logging
import pydiva2d
import numpy as np
import matplotlib.pyplot as plt
import folium
import subprocess
from importlib import reload
%matplotlib inline

/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


We set up the logger so that only *info* messages are displayed on screen.

In [2]:
logger = logging.getLogger('diva2D')
logger.setLevel(logging.INFO)

# Prepare input files and directories

In this example the input files are already created and can be found in the *data* directory.

In [3]:
datadir = './data/'
datafile = os.path.join(datadir, 'MLD1.dat')
coastfile = os.path.join(datadir, 'coast.cont')
paramfile = os.path.join(datadir, 'param.par')

We also create a directory *html* where we will save the resulting `Leaflet` maps.

In [4]:
htmldir = './html/'
if os.path.exists(htmldir):
    logger.debug("Directory {0} already exists".format(htmldir))
else:
    logger.debug("Creating directory {0}".format(htmldir))
    os.makedirs(htmldir)

## Create paths and file names

We create the paths for the Diva directories and files:

In [5]:
divadir = "/home/ctroupin/Software/DIVA/diva-4.7.1"
DivaDirs = pydiva2d.DivaDirectories(divadir)
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

INFO:diva2D:Diva main directory: /home/ctroupin/Software/DIVA/diva-4.7.1
INFO:diva2D:Creating Diva directory paths
INFO:diva2D:Diva binary directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/bin
INFO:diva2D:Diva source directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/src/Fortan
INFO:diva2D:Diva 2D directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped
INFO:diva2D:Diva 4D directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology
INFO:diva2D:Diva 4D input directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/input
INFO:diva2D:Creating Diva 2D file names and paths
INFO:diva2D:Contour file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/coast.cont
INFO:diva2D:Parameter file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/param.par
INFO:diva2D:Data file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/data.dat
INFO:diva2D:Valatxy file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D

## Read input files

Let's read the information from the input files.<br>
For each input file, we first creat an object (i.e., **Data**, **Contour** and **Param**) that will allow us to perform operations such as 
* reading, 
* writing, 
* plotting, 
* getting a description.

In [6]:
Data = pydiva2d.Diva2DData()
Data.read_from(datafile)
Contour = pydiva2d.Diva2DContours()
Contour.read_from(coastfile)
Param = pydiva2d.Diva2DParameters()
Param.read_from(paramfile)

INFO:diva2D:Creating Diva 2D data object
INFO:diva2D:Reading data from file ./data/MLD1.dat
INFO:diva2D:Creating Diva 2D contour object
INFO:diva2D:Reading contours from file ./data/coast.cont
INFO:diva2D:Creating Diva 2D parameter object
INFO:diva2D:Reading parameters from file ./data/param.par


# Make the analysis

We write the inputs to the corresponding files using the *write_to* functions available for each type of input:

In [7]:
Data.write_to(DivaFiles.data)
Param.write_to(DivaFiles.parameter)
Contour.write_to(DivaFiles.contour)

INFO:diva2D:Written data into file /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/data.dat
INFO:diva2D:Written parameters into file /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/param.par
INFO:diva2D:Number of contours: 28
INFO:diva2D:Number of contours: 28
INFO:diva2D:Written contours into file /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/coast.cont


## Generate the mesh

We simply call the **divamesh** script:

In [8]:
subprocess.Popen("./divamesh", cwd=DivaDirs.diva2d, shell=True)

and we create a new **Mesh** object using the information from the 2 files generated by the mesh generation:

In [9]:
Mesh = pydiva2d.Diva2DMesh(DivaFiles.mesh, DivaFiles.meshtopo)

INFO:diva2D:Creating Diva 2D mesh object


## Perform the analysis

We call the **divacalc** script:

In [10]:
subprocess.Popen("./divacalc", cwd=DivaDirs.diva2d, shell=True)

and load the results in a new object **Analysis**:

In [11]:
Analysis = pydiva2d.Diva2DResults(DivaFiles.result)

# Create the maps

In this example the plots are created using the Folium module.

## Initialise the projection

We will center the map on the region specified by the parameters.

In [12]:
divamap = folium.Map(location=[.5 * (Param.yori + Param.yend) , .5 * (Param.xori + Param.xend)], 
                     min_lat=Param.yori, max_lat=Param.yend, 
                     min_lon=Param.xori, max_lon=Param.xend)

## Data points

In [13]:
datapoints = folium.CircleMarker([Data.y, Data.x])
divamap.add_child(datapoints)
divamap.save(os.path.join(htmldir, 'datapoints.html'))

## Contours

In [22]:
Contour.x?

In [21]:
for lons, lats in zip(Contour.x, Contour.y):
    c = folium.PolyLine(locations=np.fliplr(np.append(contour, contour[0:1, :], axis=0)), 
                        color='orange', weight=5, opacity=0.75)
    divamap.add_child(c)
divamap.save(os.path.join(htmldir, 'contours.html'))

NameError: name 'contours' is not defined

## Finite element mesh

Need to convert the coordinates of the mesh to a geoJSON file...

## Analysis

Also need to convert the field from the analysis to a geoJSON